In [ ]:
import openai
import os
import requests
import json
from concurrent.futures import ThreadPoolExecutor
import tiktoken

# Add your own OpenAI API key
from dotenv import load_dotenv
from pathlib import Path

load_dotenv()

openai.api_key = os.getenv("AZURE_OPENAI_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future

In [ ]:
source_text = Path("YOUR_TEXT").read_text()
print(source_text)

In [ ]:
def split_text(text, max_length=1024):
    chunks = []
    words = text.split()
    current_chunk = ""
    for word in words:
        if len(current_chunk) + len(word) < max_length:
            current_chunk += f" {word}"
        else:
            chunks.append(current_chunk.strip())
            current_chunk = f"{word}"
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

In [ ]:
def generate_summary(text):
    input_chunks = split_text(text)
    output_chunks = []
    for chunk in input_chunks:
        response = openai.Completion.create(
            engine="YOUR_DEPLOYMENT_NAME",
            prompt=("This text is a chunked summary of a transcript. Create a coherent summary based on the text. Avoid repeating names and do not mention when participants join or leave the meeting" + "\n\n" + f"{chunk}"),
            temperature=0.5,
            max_tokens=1024,
            n = 1,
            stop=None
        )
        summary = response.choices[0].text.strip()
        output_chunks.append(summary)
    return " ".join(output_chunks)

In [ ]:
def summary_of_summaries(first_summary_text):
    first_summary_text = generate_summary(first_summary_text)
    response = openai.Completion.create(
        engine="YOUR_DEPLOYMENT_NAME",
        prompt=("This text is a chunked summary of summarized text. Create a coherent 500-1000 word blended summary of the key points based on the text. Do not mention transcript. Write as if you are describing what was discussed. Discuss the topics themselves and not who discussed them." + "\n\n" + f"{first_summary_text}"),
            temperature=0.5,
            max_tokens=1024,
            n = 1,
            stop=None,
        
    )
    summary_of_summary = response['choices'][0]['text']
    return summary_of_summary

In [ ]:
responses = summary_of_summaries(source_text)
print(responses)